<!--HEADER-->
[*Notas sobre modelagem da epidemia de Covid-19*](https://github.com/rmsrosa/modcovid19) / [*IM-UFRJ*](https://www.im.ufrj.br).

<!--BADGES-->
<a href="../slides/15.00.Aula-Modelos_individuais_velocidade.slides.html" target="_blank"><img align="left" src="https://img.shields.io/badge/local-slides-darkgreen" alt="localslides" title="Local Slides"></a>
&nbsp;

<!--NAVIGATOR-->
[<- Modelos individuais - convergência em redes completas](14.00.Aula-Modelos_individuais_convergencia_redes_completas.ipynb) | [Página Inicial](00.00-Pagina_Inicial.ipynb) | [Modelos individuais - reformulação da implementação ->](16.00.Aula-Modelos_individuais_reformulacao.ipynb)

---


# Modelos individuais - velocidade de processsamento

- Analisar o custo computacional de cada linha de cógido da função de passo de tempo.

- Analisar opções de aceleração de cada uma dessas linhas.

**Importando bibliotecas e definindo funções a serem usadas abaixo**

In [1]:
import datetime # date and time tools

import os, sys

import numpy as np

from numba import njit, prange
import threading

import math
from timeit import repeat

import matplotlib.pyplot as plt
import seaborn as sns

import io, base64
from IPython.display import Image, HTML

In [2]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import episiming

In [3]:
dt_string = datetime.datetime.now().strftime("%d/%b/%Y")
print(f"Atualização mais recente do kernel: {dt_string}")

Atualização mais recente do kernel: 20/May/2020


In [4]:
sns.set_style("darkgrid")

## Gargalos

### Passo de tempo

Aqui o código atual de cada passo de tempo:

In [5]:
def passo_vetorial(pop_estado, redes, redes_tx_transmissao,
                   pop_fator_tx_transmissao_c, prob_nao_recuperacao,
                   pop_posicoes, f_kernel, dt):

    num_pop = len(pop_estado)

    pop_suscetiveis = np.select([pop_estado==1], [pop_estado])

    pop_infectados = np.select([pop_estado==2], [pop_estado])/2
    
    contatos_de_risco_rs = np.zeros([len(redes_tx_transmissao), num_pop])

    for j in range(len(redes_tx_transmissao)):
        for (i,k) in redes[j].edges:
            if pop_infectados[k] and pop_suscetiveis[i]:
                contatos_de_risco_rs[j][i] += 1
            elif pop_infectados[i] and pop_suscetiveis[k]:
                contatos_de_risco_rs[j][k] += 1
               
    contatos_de_risco_c = np.array(
            [np.dot(pop_infectados,
                    f_kernel(np.linalg.norm(pop_posicoes - pop_posicoes[i], 
                             axis=1)
                            )
                    )
             for i in range(num_pop)
            ]
        )

    
    lambda_rate = ((redes_tx_transmissao * contatos_de_risco_rs).sum(axis=0) +
                   pop_fator_tx_transmissao_c * contatos_de_risco_c)
    
    prob_nao_contagio = np.exp(-dt*lambda_rate)                             

    sorteio = np.random.rand(num_pop)

    pop_novos_infectados = np.select([sorteio > prob_nao_contagio], [np.ones(num_pop)])

    sorteio = np.random.rand(num_pop)

    pop_novos_recuperados = np.select([pop_infectados * sorteio > prob_nao_recuperacao], 
                                      [np.ones(num_pop)])

    return pop_estado + pop_novos_infectados + pop_novos_recuperados

### Cálculo da população

O cálculo de `num_pop` a partir de `pop_estado` é ridiculamente rápido. De qualquer forma, podemos incluí-lo como argumento. Para vermos a diferença, podemos comparar o custo em se calcular `num_pop` e entre passar ou não como parâmetro.

In [6]:
n_pop = 10000
pop_estado = np.random.randint(low=1, high=3, size = n_pop)

def f(a):
    pass

def g(num_pop, a):
    pass

%timeit len(pop_estado)

%timeit f(pop_estado)
%timeit g(n_pop, pop_estado)

115 ns ± 14.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
123 ns ± 1.38 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
152 ns ± 2.01 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### Linhas de código com puro numpy

As linhas

```python
    pop_suscetiveis = np.select([pop_estado==1], [pop_estado])

    pop_infectados = np.select([pop_estado==2], [pop_estado])/2
    
    contatos_de_risco_rs = np.zeros([len(redes_tx_transmissao), num_pop])



    
    lambda_rate = ((redes_tx_transmissao * contatos_de_risco_rs).sum(axis=0) +
                   pop_fator_tx_transmissao_c * contatos_de_risco_c)
    
    prob_nao_contagio = np.exp(-dt*lambda_rate)                             

    sorteio = np.random.rand(num_pop)

    pop_novos_infectados = np.select([sorteio > prob_nao_contagio], [np.ones(num_pop)])

    sorteio = np.random.rand(num_pop)

    pop_novos_recuperados = np.select([pop_infectados * sorteio > prob_nao_recuperacao], 
                                      [np.ones(num_pop)])

    return pop_estado + pop_novos_infectados + pop_novos_recuperados

```

são todas tratadas diretamente pelo `numpy`, portanto, já são bastante eficientes e paralelizadas (sob condições normais do sistema).

É claro que podemos ganhar mais paralelizando o conjunto todo dessas linhas.

E veremos, também, que o `np.select` não é eficiente.

### Gargalos

Os dois gargalos, no entanto, estão no cálculo dos contatos de risco, tanto das redes fixas (residencial e de local de escola/trabalho) como da comunidade.

O problema é que ambas envolvem ciclos *(loops)* em python. E não vejo como torná-los códigos tratados diretamente pelo `numpy`.

```python
    for j in range(len(redes_tx_transmissao)):
        for (i,k) in redes[j].edges:
            if pop_infectados[k] and pop_suscetiveis[i]:
                contatos_de_risco_rs[j][i] += 1
            elif pop_infectados[i] and pop_suscetiveis[k]:
                contatos_de_risco_rs[j][k] += 1
               
    contatos_de_risco_c = np.array(
            [np.dot(pop_infectados,
                    f_kernel(np.linalg.norm(pop_posicoes - pop_posicoes[i], 
                             axis=1)
                            )
                    )
             for i in range(num_pop)
            ]
        )
```

### Custo computacional

Para ilustrar o custo computacional de cada termo acima, vamos considerar o cenário de 350 habitantes usado anteriormente.

In [7]:
cenario_pop_350 = episiming.cenarios.Pop350()

Definindo as variáveis usadas pela função 

In [8]:
dt = 1
pop_estado = cenario_pop_350.pop_estado_0
redes = cenario_pop_350.redes
redes_tx_transmissao = cenario_pop_350.redes_tx_transmissao
pop_fator_tx_transmissao_c = cenario_pop_350.pop_fator_tx_transmissao_c
prob_nao_recuperacao = np.exp(-dt*cenario_pop_350.gamma)
pop_posicoes = cenario_pop_350.pop_posicoes
f_kernel = cenario_pop_350.f_kernel

Para o cálculo do tempo, vamos definir, ainda, funções para os códigos que consideramos "gargalos".

In [9]:
def get_contatos_de_risco_rs(num_pop, pop_suscetiveis, pop_infectados, redes):

    contatos_de_risco_rs = np.zeros([len(redes), num_pop])

    for j in range(len(redes)):
        for (i,k) in redes[j].edges:
            if pop_infectados[k] and pop_suscetiveis[i]:
                contatos_de_risco_rs[j][i] += 1
            elif pop_infectados[i] and pop_suscetiveis[k]:
                contatos_de_risco_rs[j][k] += 1
    return contatos_de_risco_rs

In [10]:
def get_contatos_de_risco_c(pop_infectados, pop_posicoes):
    contatos_de_risco_c = np.array(
            [np.dot(pop_infectados,
                    f_kernel(np.linalg.norm(pop_posicoes - pop_posicoes[i], 
                             axis=1)
                            )
                    )
             for i in range(num_pop)
            ]
        )
    return contatos_de_risco_c

Agora, executamos todos as linhas da função.

In [11]:
num_pop = len(pop_estado)

pop_suscetiveis = np.select([pop_estado==1], [pop_estado])

pop_infectados = np.select([pop_estado==2], [pop_estado])/2

contatos_de_risco_rs = get_contatos_de_risco_rs(num_pop, pop_suscetiveis, pop_infectados, redes)

contatos_de_risco_c = get_contatos_de_risco_c(pop_infectados, pop_posicoes)

lambda_rate = ((redes_tx_transmissao * contatos_de_risco_rs).sum(axis=0) + pop_fator_tx_transmissao_c * contatos_de_risco_c)

prob_nao_contagio = np.exp(-dt*lambda_rate)                             

sorteio = np.random.rand(num_pop)

pop_novos_infectados = np.select([sorteio > prob_nao_contagio], [np.ones(num_pop)])

sorteio = np.random.rand(num_pop)

pop_novos_recuperados = np.select([pop_infectados * sorteio > prob_nao_recuperacao], [np.ones(num_pop)])

pop_novo_estado = pop_estado + pop_novos_infectados + pop_novos_recuperados

Com todas as variáveis já calculadas e gravadas na memória, façamos as contagens de tempo:

In [12]:
%timeit num_pop = len(pop_estado)

%timeit pop_suscetiveis = np.select([pop_estado==1], [pop_estado])

%timeit pop_infectados = np.select([pop_estado==2], [pop_estado])/2

%timeit contatos_de_risco_rs = get_contatos_de_risco_rs(num_pop, pop_suscetiveis, pop_infectados, redes)

%timeit contatos_de_risco_c = get_contatos_de_risco_c(pop_infectados, pop_posicoes)

%timeit lambda_rate = ((redes_tx_transmissao * contatos_de_risco_rs).sum(axis=0) + pop_fator_tx_transmissao_c * contatos_de_risco_c)

%timeit prob_nao_contagio = np.exp(-dt*lambda_rate)                             

%timeit sorteio = np.random.rand(num_pop)

%timeit pop_novos_infectados = np.select([sorteio > prob_nao_contagio], [np.ones(num_pop)])

%timeit sorteio = np.random.rand(num_pop)

%timeit pop_novos_recuperados = np.select([pop_infectados * sorteio > prob_nao_recuperacao], [np.ones(num_pop)])

%timeit pop_novo_estado = pop_estado + pop_novos_infectados + pop_novos_recuperados

118 ns ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
53.8 µs ± 736 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
59.5 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.96 ms ± 67.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
18.9 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.5 µs ± 373 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.79 µs ± 94.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.45 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
58.7 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.4 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
63.5 µs ± 838 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.5 µs ± 47.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Na função toda, gastamos

In [13]:
%timeit passo_vetorial(pop_estado, redes, redes_tx_transmissao, pop_fator_tx_transmissao_c, prob_nao_recuperacao, pop_posicoes, f_kernel, dt)

20.9 ms ± 511 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Observe que os cálculos além dos dos contatos de risco têm um custo negligível perto desses.

## Numba JIT

Uma opção é usar o [numba](http://numba.pydata.org/numba-doc/latest/index.html) (veja [A ~5 minute guide to Numba](http://numba.pydata.org/numba-doc/latest/user/5minguide.html)), que é um compilador "just-in-time", capaz de transformar um certo conjunto de instruções em python e em numpy diretamente em código executável, no momento da definição da função.

A partir daí, qualquer chamada à função irá executar o código compilado da função.

### Numba jit na rede de contatos

Para simplificar, vamos considerar o cenário de uma rede completa e analisar o tempo necessário para executar diferentes versões do cálculo dos contatos de risco, incluindo versões com o `numba.jit`.

Para usar o `numba.jit`, vamos importar o método `njit`, que é o `jit` com a opção `nopython = True`. Esta opção evita que o código "recaia" para o python interpretado caso não consiga compilar alguma parte. Vale lembrar, aqui, que só uma parte dos métodos, funções e objetos do python padrão e do numpy funcionam adequadamente no `numba`.

Para simplificar ainda mais, vamos assumir, como no caso da rede completa, que há somente uma rede.

In [14]:
num_pop = 60
num_infectados_0 = 6
beta = 0.5
gamma = 0.2

rede_completa = episiming.cenarios.RedeCompleta(num_pop, num_infectados_0, beta, gamma)

In [15]:
pop_estado = rede_completa.pop_estado_0
rede = rede_completa.redes[0]
pop_suscetiveis = np.select([pop_estado==1], [pop_estado])
pop_infectados = np.select([pop_estado==2], [pop_estado])/2

In [16]:
def get_contatos_de_risco_nx(num_pop, rede, pop_suscetiveis, pop_infectados):

    contatos_de_risco = np.zeros(num_pop)

    for (i,k) in rede.edges:
        if pop_infectados[k] and pop_suscetiveis[i]:
            contatos_de_risco[i] += 1
        elif pop_infectados[i] and pop_suscetiveis[k]:
            contatos_de_risco[k] += 1

    return contatos_de_risco

In [17]:
def get_contatos_de_risco_py(num_pop, pop_suscetiveis, pop_infectados):
    contatos_de_risco = np.zeros(num_pop)
    for i in range(num_pop):
        for k in range(i):
            if pop_infectados[k] and pop_suscetiveis[i]:
                contatos_de_risco[i] += 1
            elif pop_infectados[i] and pop_suscetiveis[k]:
                contatos_de_risco[k] += 1
    return contatos_de_risco

In [18]:
@njit
def get_contatos_de_risco_jit(num_pop, pop_suscetiveis, pop_infectados):
    contatos_de_risco = np.zeros(num_pop)
    for i in range(num_pop):
        for k in range(i):
            if pop_infectados[k] and pop_suscetiveis[i]:
                contatos_de_risco[i] += 1
            elif pop_infectados[i] and pop_suscetiveis[k]:
                contatos_de_risco[k] += 1
    return contatos_de_risco

In [19]:
conexoes = list(rede.edges)
def get_contatos_de_risco_cx(num_pop, conexoes, pop_suscetiveis, pop_infectados):
    contatos_de_risco = np.zeros(num_pop)
    for i, k in conexoes:
            if pop_infectados[k] and pop_suscetiveis[i]:
                contatos_de_risco[i] += 1
            elif pop_infectados[i] and pop_suscetiveis[k]:
                contatos_de_risco[k] += 1
    return contatos_de_risco

In [20]:
@njit
def get_contatos_de_risco_cx_jit(num_pop, conexoes, pop_suscetiveis, pop_infectados):
    contatos_de_risco = np.zeros(num_pop)
    for i, k in conexoes:
            if pop_infectados[k] and pop_suscetiveis[i]:
                contatos_de_risco[i] += 1
            elif pop_infectados[i] and pop_suscetiveis[k]:
                contatos_de_risco[k] += 1
    return contatos_de_risco

In [21]:
from numba.typed import List
typed_conexoes = List()
[typed_conexoes.append(x) for x in conexoes]
pass

In [22]:
%timeit get_contatos_de_risco_nx(num_pop, rede, pop_suscetiveis, pop_infectados)
%timeit get_contatos_de_risco_py(num_pop, pop_suscetiveis, pop_infectados)
%timeit get_contatos_de_risco_jit(num_pop, pop_suscetiveis, pop_infectados)
%timeit get_contatos_de_risco_cx(num_pop, conexoes, pop_suscetiveis, pop_infectados)
%timeit get_contatos_de_risco_cx_jit(num_pop, typed_conexoes, pop_suscetiveis, pop_infectados)

1.57 ms ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.08 ms ± 44.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.86 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
954 µs ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
15.2 µs ± 354 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Assim, conseguimor baixar o custo computacional (nesta máquina e neste cenário) do cálculo das conexões de risco da rede de contatos de quase $1.45 \,\texttt{ms}$ para algo da ordem de $5 \,\mu\texttt{s}$, usando `get_contatos_de_risco_jit`. Isso é uma redução dramática para menos de $0,4\%$ em comparação com o custo atual.

### Numba jit na rede da comunidade

Agora, vamos buscar reduzir o custo do cálculo do poder de infecção por conta de contatos aleatórios em toda a comunidade.

Construimos as seguintes funções com `njit` e testamos a velocidade.

In [23]:
@njit
def dist2_jit(x,y):
    return (abs(x[0] - y[0])**2 + abs(x[1]-y[1])**2)**.5

@njit
def f_kernel_jit(d):
    return 1.0/(1.0 + (d/1.0)**1.5)

@njit
def get_contatos_de_risco_c_jit(num_pop, pop_infectados, pop_posicoes):

    ret = []
    for i in range(num_pop):
        f_kernel_i = [
            f_kernel_jit(dist2_jit(pop_posicoes[j], pop_posicoes[i])) 
            for j in range(num_pop)
        ]

        produto = 0
        for j in range(num_pop):
            produto += pop_infectados[j] * f_kernel_i[j]

        ret.append(produto)
    return ret

In [24]:
pop_posicoes = rede_completa.pop_posicoes
%timeit contatos_de_risco_c = get_contatos_de_risco_c(pop_infectados, pop_posicoes)
%timeit get_contatos_de_risco_c_jit(num_pop, pop_infectados, pop_posicoes)

1.64 ms ± 57.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
310 µs ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Numba jit na seleção de suscetíveis e infectados

Após reduzir de milisegundos a microsegundos o custo computacional do cálculo dos contatos de risco na rede social, podemos pensar em acelerar as outras partes, também.

O termo mais custoso (contatos com a comunidade) deixaremos para depois. Abaixo, analisamos a aceleração do código de seleção de infectados e suscetíveis.

In [25]:
def get_estado_np_select(pop_estado, estado):
    return np.select([pop_estado==estado], [pop_estado])/estado

def get_estado_np_py(pop_estado, estado):
    return np.array([1 if e == estado else 0 for e in pop_estado])

@njit
def get_estado_jit_np_py(pop_estado, estado):
    return np.array([1 if e == estado else 0 for e in pop_estado])

@njit
def get_estado_jit_loop(pop_estado, estado):
    pop_selecionados = np.zeros_like(pop_estado)
    for j in range(len(pop_estado)):
        if pop_estado[j] == estado:
            pop_selecionados[j] = 1
    return pop_selecionados

In [26]:
pop_estado = rede_completa.pop_estado_0

%timeit get_estado_np_select(pop_estado, 1)
%timeit get_estado_np_py(pop_estado, 1)
%timeit get_estado_jit_np_py(pop_estado, 1)
%timeit get_estado_jit_loop(pop_estado, 1)

53.9 µs ± 554 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
49.8 µs ± 2.83 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.11 µs ± 21.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.05 µs ± 27.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Excelente! O `jit` reduziu sensivelmente o custo desse cálculo, também.

## Paralelização com o numba

Podemos buscar uma aceleração maior ainda implementando alguma forma de paralelização.

A seguir, analisamos duas opções.

### Paralelização com numba jit e *multi-threading*

Podemos combinar o `numba` com multiprocessamento via [threading](https://docs.python.org/3/library/threading.html).

Vamos começar com um exemplo adaptado de [Numba: Multi-threading](http://numba.pydata.org/numba-doc/dev/user/examples.html#multi-threading), para ver como ele se sai nessa máquina.

Primeiro, montamos uma função que serve de "envelope" *(wrapper)*, para paralelizar qualquer função que tenha a característica que agir em cada componente de um *array* separadamente *(element-wise)*.

In [27]:
def timefunc(correct, s, func, *args, **kwargs):
    """
    Benchmark *func* and print out its runtime.
    """
    print(s.ljust(20), end=" ")
    # Make sure the function is compiled before we start the benchmark
    res = func(*args, **kwargs)
    if correct is not None:
        assert np.allclose(res, correct), (res, correct)
    # time it
    print('{:>5.0f} ms'.format(min(repeat(lambda: func(*args, **kwargs),
                                          number=5, repeat=2)) * 1000))
    return res

def make_singlethread(inner_func):
    """
    Run the given function inside a single thread.
    """
    def func(*args):
        length = len(args[0])
        result = np.empty(length, dtype=np.float64)
        inner_func(result, *args)
        return result
    return func

def make_multithread(inner_func, numthreads):
    """
    Run the given function inside *numthreads* threads, splitting its
    arguments into equal-sized chunks.
    """
    def func_mt(*args):
        length = len(args[0])
        result = np.empty(length, dtype=np.float64)
        args = (result,) + args
        chunklen = (length + numthreads - 1) // numthreads
        # Create argument tuples for each input chunk
        chunks = [[arg[i * chunklen:(i + 1) * chunklen] for arg in args]
                  for i in range(numthreads)]
        # Spawn one thread per chunk
        threads = [threading.Thread(target=inner_func, args=chunk)
                   for chunk in chunks]
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
        return result
    return func_mt


Agora, definimos duas versões de uma função de teste, uma para usar apenas `numpy` e outra para o `numba`.

In [28]:
def func_np(a, b):
    """
    Control function using Numpy.
    """
    return np.exp(2.1 * a + 3.2 * b)

@njit('void(double[:], double[:], double[:])', nogil=True)
def inner_func_nb(result, a, b):
    """
    Function under test.
    """
    for i in range(len(result)):
        result[i] = math.exp(2.1 * a[i] + 3.2 * b[i])

In [29]:
nthreads = os.cpu_count()
size = 10**6

a = np.random.rand(size)
b = np.random.rand(size)

func_nb = make_singlethread(inner_func_nb)
func_nb_mt = [make_multithread(inner_func_nb, n+1) for n in range(nthreads)]

correct = timefunc(None, "numpy (1 thread)", func_np, a, b)

for n in range(nthreads):
    timefunc(correct, f"numba ({n+1} thread)", func_nb_mt[n], a, b)

numpy (1 thread)       100 ms
numba (1 thread)        71 ms
numba (2 thread)        39 ms
numba (3 thread)        37 ms
numba (4 thread)        34 ms


Usando `%timeit`:

In [30]:
%timeit func_np(a, b)

for n in range(nthreads):
    %timeit func_nb_mt[n](a, b)

19.6 ms ± 718 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.3 ms ± 184 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.74 ms ± 452 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.14 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.47 ms ± 634 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
@njit
def inner_func_nb_jit(a, b):
    """
    Function under test.
    """
    result = np.zeros_like(a)
    for i in range(len(result)):
        result[i] = math.exp(2.1 * a[i] + 3.2 * b[i])
    return result

@njit('void(double[:], double[:], double[:])', nogil=True)
def inner_func_nb_jit_mt_template(result, a, b):
    """
    Function under test.
    """
    for i in range(len(result)):
        result[i] = math.exp(2.1 * a[i] + 3.2 * b[i])

inner_func_nb_jit_mt = [make_multithread(inner_func_nb_jit_mt_template, j+1) 
                          for j in range(os.cpu_count())]

In [32]:
print('Tempos de np, jit, e jit_mt com 1, 2, 3 e 4 threads')
%timeit func_np(a,b)
%timeit inner_func_nb_jit(a,b)
for j in range(os.cpu_count()):
    %timeit inner_func_nb_jit_mt[j](a, b)

Tempos de np, jit, e jit_mt com 1, 2, 3 e 4 threads
18.7 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.3 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.1 ms ± 35.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.18 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.87 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.28 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ok, é claro que é possível acelerar o código paralelizando via `numba.jit`. Mas parece que ele só usou 2 *cores* adequadamente, não aproveitando que são 2 *threads* por *core* por CPU. E também há a questão do *overhead*.

Em relação a *cores* vs *threads*, veja [Optimizing CPU options - CPU cores and threads per CPU core per instance type](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/instance-optimize-cpu.html#cpu-options-supported-instances-values).

## Paralelização com numba jit e *prange*

Outra opção é via [prange](https://numba.pydata.org/numba-doc/dev/user/parallel.html#explicit-parallel-loops) *(parallel range)* do próprio `numba`, com a opção `parallel=True`.

In [33]:
@njit(parallel=False)
def range_test(a):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in range(a.shape[0]):
        s += a[i]
    return s

@njit(parallel=True)
def prange_test(a):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(a.shape[0]):
        s += a[i]
    return s

In [34]:
a = np.random.rand(100000)
%timeit range_test(a)
%timeit prange_test(a)

197 µs ± 8.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
75.8 µs ± 28 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Numba jit em paralelo na seleção de suscetíveis e infectados

Com essa paralelização em mente, podemos ver se conseguimos reduzir ainda mais os códigos de seleção de infectados e suscetíveis.

In [35]:
def make_multithread_mod(inner_func, numthreads):
    """
    Run the given function inside *numthreads* threads, splitting its
    arguments into equal-sized chunks.
    """
    def func_mt(pop_estado, estado):
        length = len(pop_estado)
        result = np.empty(length, dtype=np.float64)
        args = (result, pop_estado)
        chunklen = (length + numthreads - 1) // numthreads
        # Create argument tuples for each input chunk
        chunks = [[arg[i * chunklen:(i + 1) * chunklen] for arg in args]
                  for i in range(numthreads)]
        # Spawn one thread per chunk
        threads = [threading.Thread(target=inner_func, args=(*chunk, estado))
                   for chunk in chunks]
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
        return result
    return func_mt

In [36]:
@njit('void(double[:], double[:], int8)', nogil=True)
def get_estado_jit_mp_template(result, pop_estado, estado):
    for j in range(len(result)):
        if pop_estado[j] == estado:
            result[j] = 1
        else:
            result[j] = 0

In [37]:
get_estado_jit_mp = make_multithread_mod(get_estado_jit_mp_template, os.cpu_count())

In [38]:
numthreads = os.cpu_count()
def get_estado_jit_mp_2(pop_estado, estado):
    length = len(pop_estado)
    result = np.empty(length, dtype=np.float64)
    args = (result, pop_estado)
    chunklen = (len(pop_estado) + numthreads - 1) // numthreads
    # Create argument tuples for each input chunk
    chunks = [[arg[i * chunklen:(i + 1) * chunklen] for arg in args]
              for i in range(numthreads)]
    # Spawn one thread per chunk
    threads = [threading.Thread(target=get_estado_jit_mp_template, args=(*chunk, estado))
               for chunk in chunks]
    for chunk in chunks:
        aux = (*chunk, estado)
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    return result

In [39]:
@njit(parallel=True)
def get_estado_jit_mp_prange(pop_estado, estado):
    pop_selecionados = np.zeros_like(pop_estado)
    for j in prange(len(pop_estado)):
        if pop_estado[j] == estado:
            pop_selecionados[j] = 1
    return pop_selecionados

In [40]:
pop_estado = rede_completa.pop_estado_0

%timeit get_estado_np_select(pop_estado, 1)
%timeit get_estado_np_py(pop_estado, 1)
%timeit get_estado_jit_np_py(pop_estado, 1)
%timeit get_estado_jit_loop(pop_estado, 1)
%timeit get_estado_jit_mp(pop_estado,1)
%timeit get_estado_jit_mp_2(pop_estado,1)
%timeit get_estado_jit_mp_prange(pop_estado,1)

66.5 µs ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
52.2 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.11 µs ± 85.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.21 µs ± 299 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
456 µs ± 7.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
477 µs ± 23.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
The slowest run took 12.04 times longer than the fastest. This could mean that an intermediate result is being cached.
21.6 µs ± 25.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Ops, multi-threading funcionou nos exemplos acima mas não nesse caso. Precisamos avaliar melhor as construções acima.

<!--NAVIGATOR-->

---
[<- Modelos individuais - convergência em redes completas](14.00.Aula-Modelos_individuais_convergencia_redes_completas.ipynb) | [Página Inicial](00.00-Pagina_Inicial.ipynb) | [Modelos individuais - reformulação da implementação ->](16.00.Aula-Modelos_individuais_reformulacao.ipynb)